# Elasticsearch Quick Start

<a target="_blank" href="https://colab.research.google.com/github/elasticsearch-labs/colab-notebooks-examples/blob/notebooks-guides/search/00-quick-start.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This interactive notebook will introduce you to the very basics of getting started with simple Elasticsearch queries, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
We'll run through getting the client up and running, indexing a small data set into Elasticsearch, and performing basic searches against your data.

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Select **Create deployment**

## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to install the `elasticsearch` Python client.

In [ ]:
!pip install elasticsearch

Next we need to import the modules we need.

In [ ]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [ ]:
# Create the client instance
client = Elasticsearch(
    cloud_id="<CLOUD_ID>",
    basic_auth=("elastic", "<PASSWORD>")
)

If you're running Elasticsearch locally or self-managed, you can pass in the Elasticsearch host instead. [Read more](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later) on how to connect to Elasticsearch locally

Confirm that the client has connected with this test.

In [ ]:
print(client.info())

## Index some test data

Our client is set up and connected to our Elastic deployment.
Now we need some data to test out the basics of Elasticsearch queries.
We'll use a small index of books with the following fields:

- `title`
- `authors`
- `publish_date`
- `num_reviews`
- `publisher`

### Create index

Let's create an Elasticsearch index with the correct mappings for our test data.

In [ ]:
# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "authors": {"type": "keyword"},
            "summary": {"type": "text"},
            "publish_date": {"type": "date"},
            "num_reviews": {"type": "integer"},
            "publisher": {"type": "keyword"}
        }
    }
}

# Create the index
client.indices.create(index='book_index', body=mapping)


### Index test data

Run the following command to upload some test data, containing information about 10 popular programming books.

ℹ️ If you'd like to upload your own data from a URL, refer to the following [notebook](https://github.com/leemthompo/notebook-tests/blob/main/load-data-from-url.ipynb) for an example.

In [ ]:
books = [
    {
        "title": "The Pragmatic Programmer: Your Journey to Mastery",
        "authors": ["andrew hunt", "david thomas"],
        "summary": "A guide to pragmatic programming for software engineers and developers",
        "publish_date": "2019-10-29",
        "num_reviews": 30,
        "publisher": "addison-wesley"
    },
    {
        "title": "Python Crash Course",
        "authors": ["eric matthes"],
        "summary": "A fast-paced, no-nonsense guide to programming in Python",
        "publish_date": "2019-05-03",
        "num_reviews": 42,
        "publisher": "no starch press"
    },
    {
        "title": "Artificial Intelligence: A Modern Approach",
        "authors": ["stuart russell", "peter norvig"],
        "summary": "Comprehensive introduction to the theory and practice of artificial intelligence",
        "publish_date": "2020-04-06",
        "num_reviews": 39,
        "publisher": "pearson"
    },
    {
        "title": "Clean Code: A Handbook of Agile Software Craftsmanship",
        "authors": ["robert c. martin"],
        "summary": "A guide to writing code that is easy to read, understand and maintain",
        "publish_date": "2008-08-11",
        "num_reviews": 55,
        "publisher": "prentice hall"
    },
    {
        "title": "You Don't Know JS: Up & Going",
        "authors": ["kyle simpson"],
        "summary": "Introduction to JavaScript and programming as a whole",
        "publish_date": "2015-03-27",
        "num_reviews": 36,
        "publisher": "oreilly"
    },
    {
        "title": "Eloquent JavaScript",
        "authors": ["marijn haverbeke"],
        "summary": "A modern introduction to programming",
        "publish_date": "2018-12-04",
        "num_reviews": 38,
        "publisher": "no starch press"
    },
    {
        "title": "Design Patterns: Elements of Reusable Object-Oriented Software",
        "authors": ["erich gamma", "richard helm", "ralph johnson", "john vlissides"],
        "summary": "Guide to design patterns that can be used in any object-oriented language",
        "publish_date": "1994-10-31",
        "num_reviews": 45,
        "publisher": "addison-wesley"
    },
    {
        "title": "The Clean Coder: A Code of Conduct for Professional Programmers",
        "authors": ["robert c. martin"],
        "summary": "A guide to professional conduct in the field of software engineering",
        "publish_date": "2011-05-13",
        "num_reviews": 20,
        "publisher": "prentice hall"
    },
    {
        "title": "JavaScript: The Good Parts",
        "authors": ["douglas crockford"],
        "summary": "A deep dive into the parts of JavaScript that are essential to writing maintainable code",
        "publish_date": "2008-05-15",
        "num_reviews": 51,
        "publisher": "oreilly"
    },
    {
        "title": "Introduction to the Theory of Computation",
        "authors": ["michael sipser"],
        "summary": "Introduction to the theory of computation and complexity theory",
        "publish_date": "2012-06-27",
        "num_reviews": 33,
        "publisher": "cengage learning"
    },
]

    actions = []
    for book in books:
        actions.append({"index": {"_index": "book_index"})
        actions.append(doc)

    client.bulk(index="book_index", operations=actions)


## Aside: Pretty printing Elasticsearch responses

Your API calls will return hard-to-read nested JSON.
We'll create a little function called `pretty_response` to return nice, human-readable outputs from our examples.

In [ ]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nScore: {score}")
        print(pretty_output)

## Simple queries

Let's start by looking at simple queries which search for a particular value in a particular field.

### `match` query

The match query is the standard query for performing a full-text search.
Returns documents that match a provided text, number, date or boolean value.
The provided text is [analyzed](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-overview.html#analysis-overview) before matching.

For multiple word queries, Elasticsearch searches for each word using OR logic.
So 'deep dive' and 'deep drive' will still return a match in our index.

In [ ]:
response = client.search(index="book_index",
              query={
                  'match': {
                      'summary': 'deep dive'
                  }})

pretty_response(response)

if you need to match the query on multiple fields, you can use multi_match

In [ ]:
response = client.search(
   query= {
            "multi_match": {
                "query": "javascript",
                "fields": ["summary", "title"],
            }
        }
)
pretty_response(response)

### `term` query

Returns documents that contain an exact term in a provided field.

You can use the term query to find documents based on a precise value such as a price, a product ID, or a username.

⚠️ Avoid using the `term` query for [text](https://www.elastic.co/guide/en/elasticsearch/reference/current/text.html) fields, because these fields are modified by analysis.
To search text field values, use the `match` query instead.

Let's say we want to search for an exact book ID:


In [ ]:
query =  {
        "term": {
            "_id": {
                "value": "_eTh_IgBM7k8MFjP03ir",
                "boost": 1.0
            }
        }
    }

response = client.search(query=query, index="book_index")
pretty_response(response)

Note that if you just searched for a substring like `gBM7k8` here, you would not get a match!
Use `term` queries to match full, exact values on non-`text` fields.

### `range` query

Returns documents that contain terms within a provided range.
Let's say we want to find all books published between January 1st 2019 and January 1st 2020.

In [ ]:
query =  {
        "range": {
            "publish_date": {
                "gte": "2019-01-01", # gte = greater than or equal to
                "lte": "2020-01-01", # lte = less than or equal to
            }
        }
    }

response = client.search(query=query, index="book_index")
pretty_response(response)